In [3]:
import tensorflow as tf
import os
import time
from datetime import datetime

In [4]:
class Generator:
    def __init__(self, depths=[1024, 512, 256, 128], s_size=4):
        #Change for greyscale changed from 3 to 1
        self.depths = depths + [1]
        self.s_size = s_size
        self.reuse = False
        print('self.depths', depths)

    def __call__(self, inputs, training=False):
        inputs = tf.convert_to_tensor(inputs)
        print('Inside Generator Inputs', inputs)
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('reshape'):
                outputs = tf.layers.dense(inputs, self.depths[0] * self.s_size * self.s_size)
                outputs = tf.reshape(outputs, [-1, self.s_size, self.s_size, self.depths[0]])
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            # deconvolution (transpose of convolution) x 4
            with tf.variable_scope('deconv1'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv2'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv3'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv4'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
            # output images
            with tf.variable_scope('tanh'):
                outputs = tf.tanh(outputs, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self, depths=[64, 128, 256, 512]):
        #Change for greyscale made 3 to 1
        self.depths = [1] + depths
        self.reuse = False

    def __call__(self, inputs, training=False):
        def leaky_relu(x, leak=0.2, name=''):
            return tf.maximum(x, x * leak, name=name)
        outputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('classify'):
                batch_size = outputs.get_shape()[0].value
                reshape = tf.reshape(outputs, [batch_size, -1])
                outputs = tf.layers.dense(reshape, 2, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [ ]:
class DCGAN:
    def __init__(self,
                 batch_size=16, s_size=4, z_dim=100,
                 g_depths=[1024, 512, 256, 128],
                 d_depths=[64, 128, 256, 512]):
        self.batch_size = batch_size
        self.s_size = s_size
        self.z_dim = z_dim
        self.g = Generator(depths=g_depths, s_size=self.s_size)
        self.d = Discriminator(depths=d_depths)
        self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)

    def loss(self, traindata):
        """build models, calculate losses.
        Args:
            traindata: 4-D Tensor of shape `[batch, height, width, channels]`.
        Returns:
            dict of each models' losses.
        """
        print('self.z is', self.z)
        generated = self.g(self.z, training=True)
        g_outputs = self.d(generated, training=True)
        t_outputs = self.d(traindata, training=True)
        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=t_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.zeros([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def train(self, losses, learning_rate=0.00002, beta1=0.5):
        """
        Args:
            losses dict.
        Returns:
            train op.
        """
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name='train')

    def sample_images(self, row=8, col=8, inputs=None):
        if inputs is None:
            inputs = self.z
        images = self.g(inputs, training=True)
        images = tf.image.convert_image_dtype(tf.div(tf.add(images, 1.0), 2.0), tf.uint8)
        images = [image for image in tf.split(images, self.batch_size, axis=0)]
        rows = []
        for i in range(row):
            rows.append(tf.concat(images[col * i + 0:col * i + col], 0))
        image = tf.concat(rows, 1)
        return tf.image.encode_png(tf.squeeze(image, [0]))

In [5]:

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('logdir', 'logdir/',
                           """logs and checkpoint data are saved in this directory.""")
tf.app.flags.DEFINE_string('images_dir', 'images/',
                           """Images generated by the generator are saved in this directory""")
tf.app.flags.DEFINE_string('data_dir', 'data',
                           """Path to read the data from. The data has to be in .tfrecord format.""")
tf.app.flags.DEFINE_integer('num_examples_per_epoch_for_train', 5000,
                            """Number of examples per epoch""")
tf.app.flags.DEFINE_integer('max_steps', 90001,
                            """Maximum number of steps to run the training.""")

#Size of input images: INPUT_IMAGE_SIZE X INPUT_IMAGE_SIZE
INPUT_IMAGE_SIZE = 164
#Defining cropped image size
#Cropping the input images to the size: crop_image_size X crop_image_size
CROP_IMAGE_SIZE = 164

#Function used to read the input greyscale images from the .tfrecord files present in the data_dir.
def inputs(batch_size, s_size):
    print('batch_size:', batch_size)
    print('s_size', s_size)
    print('path is:', os.path.join(FLAGS.data_dir))
    #Picking all the files ending in .tfrecord
    files = [os.path.join(FLAGS.data_dir, f) for f in os.listdir(FLAGS.data_dir) if f.endswith('.tfrecord')]
    print(files)
    #Using string_input_producer() to output the files into a queue which can then be used for an input pipeline. 
    fqueue = tf.train.string_input_producer(files)
    print(fqueue)
    #Using the TFRecordReader to output records from a .tfrecord file.
    reader = tf.TFRecordReader()
    #Using the read method of the TFRecordReader() class to read the next record from the queue. 
    _, value = reader.read(fqueue)
    #From the value extracting a particular feature with key 'image/encoded'.
    features = tf.parse_single_example(value, features={'image/encoded': tf.FixedLenFeature([], tf.string)})
    #change for greyscale changed from 3 to 1 chanels value. 
    #Decoding the feature as an .png image and casting it as an image. Since we want to retrieve a greyscale image the number of channels=1
    image = tf.cast(tf.image.decode_png(features['image/encoded'], channels=1), tf.float32)
    # Resizing the image to the size of the cropped image
    image = tf.image.resize_image_with_crop_or_pad(image, CROP_IMAGE_SIZE, CROP_IMAGE_SIZE)
    image = tf.image.random_flip_left_right(image)

    min_queue_examples = FLAGS.num_examples_per_epoch_for_train
    # shuffle the images into batchs of batch sizes.
    images = tf.train.shuffle_batch( [image], batch_size=batch_size, capacity=min_queue_examples + 3 * batch_size, min_after_dequeue=min_queue_examples)
    print('Images are:', images)
    tf.summary.image('images', images)
    return tf.subtract(tf.div(tf.image.resize_images(images, [s_size * 2 ** 4, s_size * 2 ** 4]), 127.5), 1.0)


def main(argv=None):
    #Creating an object of the DCGAN
    dcgan = DCGAN(s_size=10)
    #Call to function to read data in .tfrecord format
    traindata = inputs(dcgan.batch_size, dcgan.s_size)
    print('Train data', traindata)
    # Calculating the losses
    losses = dcgan.loss(traindata)
    # Extracting the Generator and Discrimintor loss
    tf.summary.scalar('g loss', losses[dcgan.g])
    tf.summary.scalar('d loss', losses[dcgan.d])
    #Minimize the Generator and the Discrimintor losses
    train_op = dcgan.train(losses)
    summary_op = tf.summary.merge_all()

    #Creating objects to save the generator and discriminator states
    g_saver = tf.train.Saver(dcgan.g.variables)
    d_saver = tf.train.Saver(dcgan.d.variables)
    #Defining the directory to store the generator check points
    g_checkpoint_path = os.path.join(FLAGS.logdir, 'gckpt/')
    print('G checkpoint path: ', g_checkpoint_path)
    ##Defining the directory to store the discriminator check points
    d_checkpoint_path = os.path.join(FLAGS.logdir, 'dckpt/')
    print('D Checkpoint Path:',d_checkpoint_path)

    if not os.path.exists(g_checkpoint_path):
        os.makedirs(g_checkpoint_path)

    if not os.path.exists(d_checkpoint_path):
        os.makedirs(d_checkpoint_path)

    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(FLAGS.logdir, graph=sess.graph)
        newStepNo = 0
        # restore or initialize generator
        sess.run(tf.global_variables_initializer())

        gckpt = tf.train.get_checkpoint_state(g_checkpoint_path)
        if gckpt and gckpt.model_checkpoint_path:
            g_saver.restore(sess, gckpt.model_checkpoint_path)
            print('Model restored from ' + gckpt.model_checkpoint_path)
            newStepCheck = gckpt.model_checkpoint_path
            newStepNo = int(newStepCheck.split('-')[1])
        #if os.path.exists(g_checkpoint_path):
            #print('restore variables for G:')
            #for v in dcgan.g.variables:
                #print('  ' + v.name)
            #g_saver.restore(sess, ckpt.model_checkpoint_path)
            #g_saver.restore(sess, g_checkpoint_path)
        #if os.path.exists(d_checkpoint_path):
            #print('restore variables for D:')
            #for v in dcgan.d.variables:
              #  print('  ' + v.name)
            #d_saver.restore(sess, d_checkpoint_path)

        dckpt = tf.train.get_checkpoint_state(d_checkpoint_path)
        if dckpt and dckpt.model_checkpoint_path:
            d_saver.restore(sess, dckpt.model_checkpoint_path)
            print('Model restored from ' + dckpt.model_checkpoint_path)

        # setup for monitoring
        sample_z = sess.run(tf.random_uniform([dcgan.batch_size, dcgan.z_dim], minval=-1.0, maxval=1.0))
        images = dcgan.sample_images(1, 1, inputs=sample_z)

        # start training
        tf.train.start_queue_runners(sess=sess)

        #for step in range(FLAGS.max_steps):
        while newStepNo <= FLAGS.max_steps:
            start_time = time.time()
            _, g_loss, d_loss = sess.run([train_op, losses[dcgan.g], losses[dcgan.d]])
            duration = time.time() - start_time
            print('{}: step {:5d}, loss = (G: {:.8f}, D: {:.8f}) ({:.3f} sec/batch)'.format(
                datetime.now(), newStepNo, g_loss, d_loss, duration))

            # save generated images
            if newStepNo % 100 == 0:
                # summary
                summary_str = sess.run(summary_op)
                summary_writer.add_summary(summary_str, newStepNo)
                # sample images
                filename = os.path.join("C:\\Users\\sedkilic\\immages", '%05d.png' % newStepNo)
                with open(filename, 'wb') as f:
                    f.write(sess.run(images))
            # save variables
            if newStepNo % 500 == 0:
                g_saver.save(sess, g_checkpoint_path + 'g.ckpt', global_step=newStepNo)
                print('Save mode for G in checkpoint_path: ', g_checkpoint_path)
                d_saver.save(sess, d_checkpoint_path + 'd.ckpt', global_step=newStepNo)
                print('Save mode for D in checkpoint_path: ', d_checkpoint_path)
                #g_saver.save(sess, g_checkpoint_path, global_step=step)
                #d_saver.save(sess, d_checkpoint_path, global_step=step)
            newStepNo = newStepNo+1


if __name__ == '__main__':
    tf.app.run()

self.depths [1024, 512, 256, 128]
batch_size: 16
s_size 10
path is: data
['data\\gun_train_00000-of-00002.tfrecord', 'data\\gun_train_00001-of-00002.tfrecord', 'data\\gun_validation_00000-of-00002.tfrecord', 'data\\gun_validation_00001-of-00002.tfrecord']
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `

SystemExit: 

C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
Exception in thread QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue:
Traceback (most recent call last):
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\training\queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, 

ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[{{node input_producer/input_producer_EnqueueMany}} = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/RandomShuffle)]]


Exception in thread QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\training\queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "C:\Users\sedkilic\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledErr